# Wildfires on the Beach: The Woolsey Fire 

* Over two weeks in November 2018, the Woolsey Fire burned nearly 100,000 acres in the Santa Monica Mountains north of Malibu, California (Fig. 1). Three people were killed in the fire and over 1,600 structures were destroyed. Two other fires that ignited the same day as the Woolsey Fire caused widespread damage throughout California.

* The Santa Ana season (Fall in Califonia), is prime time large and destructive fires. The Woolsey Fire happened in a location conducive for maximum growth; an area with little recent fire history (but history of fire going back to the 1930s) and only the beach to stop it.



![The Woolsey Fire - flames above scenic Highway 1](https://www.redzone.co/wp-content/uploads/2019/02/woolsey-fire-near-the-shore.jpg)
> Image source: https://www.redzone.co/2019/02/07/woolsey-fire-in-retrospect/


In [1]:
import logging
import os
import pathlib
import subprocess
import warnings
from glob import glob

import earthpy.appeears as etapp
import folium
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray as rx
import pandas as pd
import rioxarray as rxr
import xarray as xr

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarnings coming from hvplot
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Generating the path to the project directory
project_direc = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'woolsey-fire')

# Create the project directory
os.makedirs(project_direc, exist_ok=True)
project_direc

'/home/jovyan/earth-analytics/data/woolsey-fire'

In [3]:
# Fetching historic fire perimeter data for the "WOOLSEY" incident

woolsey_gdf = gpd.read_file(
    "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/"
    "Historic_Geomac_Perimeters_2018/FeatureServer/0/query"
    "?where=incidentname%20%3D%20'WOOLSEY'%20AND%20latest%20%3D%20'Y'"
    "&outFields=*&outSR=4326&f=json"
)
woolsey_gdf

,OBJECTID,agency,comments,active,mapmethod,datecurrent,uniquefireidentifier,fireyear,incidentname,pooownerunit,...,incomplex,complexfirecode,mergeid,latest,modifiedon,createdon,temp,shape__Area,shape__Length,geometry
0,1198,Local Agency,,N,Mixed Methods,1542499200000,2018-CAVNC-091023,2018,WOOLSEY,CAVNC,...,N,,,Y,None,None,None,5.738505e+08,250299.909321,"POLYGON ((-118.98273 34.11239, -118.98432 34.1..."


In [4]:
# Malibu, CA locations
# From https://www.latlong.net/place/malibu-ca-usa-32013.html. 
malibu_lat = 34.031246
malibu_lon = -118.788193

# Create a Folium map object
m = folium.Map(
    location=[malibu_lat, malibu_lon], 
    zoom_start=10.49, 
    scrollWheelZoom=False,
    tiles='Stamen Watercolor')

# Convert the GeoDataFrame to GeoJSON format
malibu_geojson = woolsey_gdf.to_json()

# Add the GeoJSON data to the map as a GeoJson layer
malibu_style = {"fillColor" : "red",
    "color": "black",
    "weight" : 2,
    "dashArray": "5,5",
    }
folium.GeoJson(
    malibu_geojson, 
    style_function=lambda feature: malibu_style
    ).add_to(m)

# Add a marker for Malibu, CA
folium.Marker(location=[malibu_lat, malibu_lon]).add_to(m)

# Display the map
m

In [5]:
  # Initialize AppeearsDownloader for MODIS NDVI data
  ndvi_downloader = etapp.AppeearsDownloader(
    download_key='modis-ndvi',
    ea_dir=project_direc,
    product='MYD13Q1.061',
    layer='_250m_16_days_NDVI',
    start_date='06-01',
    end_date='08-31', 
    recurring=True,
    year_range=[2017, 2022],
    polygon=woolsey_gdf
  )

  # Download files if the download directory does not exist
  if not os.path.exists(ndvi_downloader.data_dir):
    ndvi_downloader.download_files()

In [6]:
ndvi_path_list = glob(os.path.join(
    ndvi_downloader.data_dir, '*', '*NDVI*.tif'))

In [7]:
doy_start = -19
doy_end = -12
scale_factor = 10000
ndvi_da_list = []
for ndvi_path in ndvi_path_list:
    # Get date from file name
    doy = ndvi_path[doy_start:doy_end]
    date = pd.to_datetime(doy, format='%Y%j')

    # Open dataset
    da = rxr.open_rasterio(ndvi_path, masked=True).squeeze()

    # Prepare to concatenate: Add date dimension and clean up metadata
    da = da.assign_coords({'date': date})
    da = da.expand_dims({'date': 1})
    da.name = 'NDVI'

    # Divide by scale factor
    da = da / scale_factor
    # Add the DataArray to the end of the accumulator list
    ndvi_da_list.append(da)

In [8]:
ndvi_ds = xr.combine_by_coords(ndvi_da_list, coords=['date'])

In [9]:
dndvi_da = (ndvi_ds
    .sel(date='2020')
    .mean('date')
    .NDVI
 - ndvi_ds
    .sel(date='2017')
    .mean('date')
    .NDVI
)

(dndvi_da.hvplot(cmap='PiYG'))

:Image   [y,x]   (NDVI)

In [10]:
# Get the area outside the fire boundary
out_gdf = (
    gpd.GeoDataFrame(geometry=woolsey_gdf.envelope)
    .overlay(woolsey_gdf, how="difference")
)
out_gdf

,geometry
0,"MULTIPOLYGON (((-118.99636 34.00540, -118.9963..."


In [11]:
# Clip the data array to the boundaries both inside and outside the fire
# boundary
ndvi_fire_ds = ndvi_ds.rio.clip(woolsey_gdf.geometry)
ndvi_out_ds = ndvi_ds.rio.clip(out_gdf.geometry)

In [12]:
ndvi_fire_df = (
    ndvi_fire_ds
    .groupby(ndvi_fire_ds.date.dt.year)
    .mean(...)
    .NDVI
    .to_dataframe()[['NDVI']])
ndvi_out_df = (
    ndvi_out_ds
    .groupby(ndvi_out_ds.date.dt.year)
    .mean(...)
    .NDVI
    .to_dataframe()[['NDVI']])

ndvi_fire_df, ndvi_out_df

(          NDVI
 year          
 2017  0.458010
 2018  0.429461
 2019  0.400919
 2020  0.406328
 2021  0.325053
 2022  0.390789,
           NDVI
 year          
 2017  0.398876
 2018  0.380254
 2019  0.416327
 2020  0.405906
 2021  0.354659
 2022  0.381206)

In [13]:

#take the difference of the NDVI  outside and inside the fire boundaries 
(ndvi_fire_df-ndvi_out_df).hvplot(
    xlabel ='Year', ylabel='dNDVI',
    title='Difference in NDVI between inside and outside Woolsey fire boundary'
)

:Curve   [year]   (NDVI)

### Vegetation slow to recover after the Woolsey Fire

* The southern coast of California has a relatively low NDVI and the natural landscape of the Santa Monica Mountains is dominated by diverse species of woody shrubs, also known as chaparral. Wildfire is a necessary phase in the life cycle of chaparral plants. Despite the fact that wildfire is necessity for charral, the slow regrowth of vegetation could be in part due to the severe drought with in the state during that time period.